In [21]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [120]:
# Import system helpers
import os

# Import TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Import helper libraries
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt

In [121]:
# Convert .mat to Python dict with Scipy.io

sc_devkit = '/Users/katerina/Workspace/visual_census/data/devkit'

cars_meta = scipy.io.loadmat(os.path.join(sc_devkit, 'cars_meta.mat'))['class_names']
cars_annos = scipy.io.loadmat(os.path.join(sc_devkit, 'cars_annos.mat'))['annotations']

In [135]:
# Extract Car Metadata from dictionary to an array

car_makes = []
for idx, vehicle in enumerate(cars_meta[0]):
    car_makes.append([vehicle[0],vehicle[0].split(' ')[0], vehicle[0].split(' ')[-1]])
car_makes = pd.DataFrame(car_makes, columns=['full_label', 'mnfr', 'year'])

In [130]:
# Extract Car Labels from dictionary to a list
# List used because both int (label) and str (img id) required

cars_labels = []
cars_annotations = []
for idx, anno in enumerate(cars_annos[0]):
    cars_labels.append([anno[0][0], anno[5][0][0]])
    cars_annotations.append([anno[0][0], anno[1][0][0], anno[2][0][0], anno[3][0][0], anno[4][0][0], anno[5][0][0]])
cars_annotations = pd.DataFrame(cars_bb, columns=['img_path', 'bb0', 'bb1', 'bb2', 'bb3', 'label'])

In [140]:
#

In [141]:
#